# Initial AI Agent PPO

In [ ]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py


from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray import tune
import json
import numpy as np

ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 1024
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2


def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

ray.init()

#agent = ppo.PPO(env="gt-rtgym-env-v1", config=ppoconfig)

algo = (
    PPOConfig()
    #SACConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        #batch_mode="truncate_episodes",
        #rollout_fragment_length=128
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=0.0003,
            #lambda_=0.95,
            #gamma=0.99,
            sgd_minibatch_size=128,
            train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 500
results = []
episode_data = []
episode_json = []


for n in range(N):
    result = algo.train()
    results.append(result)
  
    episode = {
        "n": n,
        "episode_reward_mean": result["episode_reward_mean"], 
        "episode_reward_max":  result["episode_reward_max"],  
        "episode_len_mean":    result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    
    print('Loop: ', n, 'Max reward: ', {episode["episode_reward_max"]})
    
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
import json
import numpy as np

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 256
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

# Use the Algorithm's `from_checkpoint` utility to get a new algo instance
# that has the exact same state as the old one, from which the checkpoint was
# created in the first place:
loadedAlgo = Algorithm.from_checkpoint(path_to_checkpoint)

In [ ]:

N = 300
results = []
episode_data = []
episode_json = []


for n in range(N):
    result = loadedAlgo.train()
    results.append(result)
  
    episode = {
        "n": n,
        "episode_reward_mean": result["episode_reward_mean"], 
        "episode_reward_max":  result["episode_reward_max"],  
        "episode_len_mean":    result["episode_len_mean"],
    }

    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    
    print('Loop: ', n, 'Max reward: ', {episode["episode_reward_max"]})

In [ ]:
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

In [ ]:
ray.shutdown()

# SAC Current Developing

In [2]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.1
my_config["start_obs_capture"] = 0.1
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 128
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)
ray.init()

algo = (
    #PPOConfig()
    SACConfig()
    .resources(
        num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        # batch_mode="truncate_episodes",
        # rollout_fragment_length=128,
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            #train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 50

for n in range(N):
    result = algo.train()
    if n % 5 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

2023-05-08 23:34:47,275	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=10332) GT Real Time instantiated
(RolloutWorker pid=10332) GT AI Server instantiated for rtgym
(RolloutWorker pid=10332) starting up on localhost port 9999
(RolloutWorker pid=10332) Waiting for a connection
(RolloutWorker pid=10332) Connection from ('127.0.0.1', 49279)
(RolloutWorker pid=10332) default SAC action


2023-05-08 23:35:02,844	INFO trainable.py:172 -- Trainable.setup took 13.980 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-08 23:35:02,846	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=10332) reset triggered
(RolloutWorker pid=10332) reload save


(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16870019999999997
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


...

(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.430445600000013
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3343477999999891
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3332873000000234
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTA

(RolloutWorker pid=10332) reset triggered
(RolloutWorker pid=10332) reload save


(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.8538044999999954
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.37082430000000954
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.36751160000000027
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\

KeyboardInterrupt: 

(RolloutWorker pid=10332) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3249324000000229
(RolloutWorker pid=10332)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


# A3C Method

In [2]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 512
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)
ray.init()

algo = (
    #PPOConfig()
    #SACConfig()
    A3CConfig()
    .resources(
        #num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        # batch_mode="truncate_episodes",
        # rollout_fragment_length=128,
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            #train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 2000

for n in range(N):
    result = algo.train()
    if n % 10 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

2023-05-13 00:13:02,460	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(RolloutWorker pid=3128) GT Real Time instantiated
(RolloutWorker pid=3128) GT AI Server instantiated for rtgym
(RolloutWorker pid=3128) starting up on localhost port 9999
(RolloutWorker pid=3128) Waiting for a connection


2023-05-13 00:13:17,479	INFO trainable.py:172 -- Trainable.setup took 13.503 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-13 00:13:17,481	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=3128) Connection from ('127.0.0.1', 51697)
(RolloutWorker pid=3128) default SAC action
(RolloutWorker pid=3128) reset triggered
(RolloutWorker pid=3128) reload save


(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3142609999999273
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6693763999999938
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Saved 0


(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.362116499999992
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.4873443999999161
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.5332982999999913
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\

(RolloutWorker pid=3128) reset triggered
(RolloutWorker pid=3128) reload save


(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.8997226000000254
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.6548489000003457
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=3128) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 1.548070000000024
(RolloutWorker pid=3128)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


KeyboardInterrupt: 

In [ ]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.algorithms.a3c import A3CConfig
import json
import numpy as np
from time import sleep

ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
#my_config["ep_max_length"] = 512
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2


def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=my_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", env_creator)

loadedAlgo = Algorithm.from_checkpoint(path_to_checkpoint)

loadedAlgo.train()

In [ ]:
import ray
from ray import air, tune
ray.shutdown()
ray.init()


config = PPOConfig().training(lr=tune.grid_search([0.01, 0.001, 0.0001])).environment(env="CartPole-v1")

tuner = tune.Tuner(
    "PPO",
    run_config=air.RunConfig(
        stop={"episode_reward_mean": 150},
    ),
    param_space=config,
)

tuner.fit()

# PPO with checkpoints

In [1]:
# to read later: https://raw.githubusercontent.com/ray-project/ray/master/rllib/examples/unity3d_env_local.py
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 512
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=env_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

ray.init()

algo = (
    PPOConfig()
    .resources(
        #num_gpus=1
        )
    .rollouts(
        num_rollout_workers=1,
        enable_connectors=True,
        # batch_mode="truncate_episodes",
        # rollout_fragment_length=128,
        )
    .framework("torch")
    .environment(
        env="gt-rtgym-env-v1",
        disable_env_checking=True,
        render_env=False,
        )
    .training(
            #lr=tune.grid_search([0.01, 0.001, 0.0001])
            #lambda_=0.95,
            #gamma=0.99,
            #sgd_minibatch_size=128,
            #train_batch_size=1024,
            #num_sgd_iter=8,
            #clip_param=0.2,
            #model={"fcnet_hiddens": [12, 12]},
        )
    .build()
)

N = 500

for n in range(N):
    result = algo.train()
    if n % 1 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

ray.shutdown()

2023-05-13 23:47:03,333	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-05-13 23:47:05,035	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=20424) GT Real Time instantiated
(RolloutWorker pid=20424) GT AI Server instantiated for rtgym
(RolloutWorker pid=20424) starting up on localhost port 9999
(RolloutWorker pid=20424) Waiting for a connection


2023-05-13 23:47:25,264	INFO trainable.py:172 -- Trainable.setup took 20.230 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-13 23:47:25,265	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=20424) Connection from ('127.0.0.1', 54862)
(RolloutWorker pid=20424) default SAC action
(RolloutWorker pid=20424) reset triggered
(RolloutWorker pid=20424) reload save
(RolloutWorker pid=20424) reset triggered
(RolloutWorker pid=20424) reload save


(RolloutWorker pid=20424) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16650020000062682
(RolloutWorker pid=20424)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=20424) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 19.085009000001435
(RolloutWorker pid=20424)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=20424) reset triggered
(RolloutWorker pid=20424) reload save


(RolloutWorker pid=20424) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16042960000001472
(RolloutWorker pid=20424)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=20424) reset triggered
(RolloutWorker pid=20424) reload save


(RolloutWorker pid=20424) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.15158170000145788
(RolloutWorker pid=20424)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


KeyboardInterrupt: 

# Continue Training

In [1]:
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium
import ray
from ray import tune, air
from ray.rllib.algorithms import ppo
from ray.rllib.algorithms.ppo import PPOConfig
import json
import numpy as np
from time import sleep
ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["ep_max_length"] = 512
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=env_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way

ray.init()

from ray.rllib.algorithms.algorithm import Algorithm
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-13_00-31-13_kmaa6ii/checkpoint_000161")

2023-05-13 23:51:00,859	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-05-13 23:51:02,454	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=23896) GT Real Time instantiated
(RolloutWorker pid=23896) GT AI Server instantiated for rtgym
(RolloutWorker pid=23896) starting up on localhost port 9999
(RolloutWorker pid=23896) Waiting for a connection


2023-05-13 23:51:13,722	INFO trainable.py:172 -- Trainable.setup took 11.269 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2023-05-13 23:51:13,724	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=23896) Connection from ('127.0.0.1', 55182)
(RolloutWorker pid=23896) default SAC action


In [2]:
N = 50

for n in range(N):
    result = algo.train()
    if n % 5 == 0:
        print("Saved", n)
        algo.save()
path_to_checkpoint = algo.save()
print(
    "An Algorithm checkpoint has been created inside directory: "
    f"'{path_to_checkpoint}'."
)

(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save


(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.3807117000000204
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save


(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1701696000001789
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.1677496000014429
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save
(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save


(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16404320000116002
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")
(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.14084650000107501
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save
(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save


(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16736340000105088
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


(RolloutWorker pid=23896) reset triggered
(RolloutWorker pid=23896) reload save


(RolloutWorker pid=23896) c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 0.16450739999660868
(RolloutWorker pid=23896)   warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Test Agent

In [2]:
from ray.rllib.algorithms.algorithm import Algorithm
import ray
from myRTClass import MyGranTurismoRTGYM, DEFAULT_CONFIG_DICT
import gymnasium

ray.shutdown()

my_config = DEFAULT_CONFIG_DICT
my_config["interface"] = MyGranTurismoRTGYM
my_config["time_step_duration"] = 0.05
my_config["start_obs_capture"] = 0.05
my_config["time_step_timeout_factor"] = 1.0
my_config["act_buf_len"] = 3
my_config["reset_act_buf"] = False
my_config["benchmark"] = True
my_config["benchmark_polyak"] = 0.2

def env_creator(env_config):
  env = gymnasium.make("real-time-gym-v1", config=env_config)
  return env  # return an env instance

from ray.tune.registry import register_env
register_env("gt-rtgym-env-v1", lambda config: env_creator(my_config)) # better way
#register_env("gt-rtgym-env-v1", env_creator)



ray.init()

# env = gymnasium.make("real-time-gym-v1")
algo = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-13_00-31-13_kmaa6ii/checkpoint_000161")


2023-05-13 23:42:33,654	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-05-13 23:42:35,210	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=20848) GT Real Time instantiated
(RolloutWorker pid=20848) GT AI Server instantiated for rtgym
(RolloutWorker pid=20848) starting up on localhost port 9999
(RolloutWorker pid=20848) Waiting for a connection


2023-05-13 23:42:42,528	WARNING util.py:67 -- Install gputil for GPU system monitoring.


(RolloutWorker pid=20848) Connection from ('127.0.0.1', 54273)
(RolloutWorker pid=20848) default SAC action


In [3]:
algo.stop()

In [4]:
policy = algo.get_policy()
model = policy.model

In [5]:
new_config = {
    # Indicate that the Algorithm we setup here doesn't need an actual env.
    "env": None,
    "observation_space": model.obs_space,
    "action_space": model.action_space,
    # ...
}

In [6]:
algo.config = new_config

In [1]:
env = gymnasium.make("real-time-gym-v1", config=my_config)

NameError: name 'gymnasium' is not defined

In [ ]:
episode_reward = 0
terminated = truncated = False
obs, info = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward

In [7]:
import gymnasium.spaces as spaces
import numpy as np

def get_observation_space():
    eClutch = spaces.Box(low=0, high=3, shape=(1,), dtype='float32')
    eSpeed = spaces.Box(low=0, high=10000, shape=(1,), dtype='float32')
    eGear =  spaces.Box(low=0, high=6, shape=(1,), dtype='float32')
    vSpeed = spaces.Box(low=0, high=500, shape=(1,), dtype='float32')
    rState = spaces.Box(low=0, high=5, shape=(1,), dtype='float32')
    vDir = spaces.Box(low=0, high=3, shape=(1,), dtype='float32')
    return spaces.Tuple((rState, eClutch, eSpeed, eGear, vSpeed, vDir))
    
def get_action_space():
    #return spaces.Box(low=-1.0, high=1.0, shape=(3,))
    return spaces.Box(low=np.array([0.0, 0.0, -1.0]), high=np.array([1.0, 1.0, 1.0]), dtype='float64')

In [8]:
new_config2 = {
    # Indicate that the Algorithm we setup here doesn't need an actual env.
    "env": None,
    "observation_space": get_observation_space(),
    "action_space": get_action_space(),
    # ...
}

In [9]:
env = gymnasium.make("real-time-gym-v1", config=my_config)

GT Real Time instantiated
GT AI Server instantiated for rtgym
starting up on localhost port 9999
Waiting for a connection
Connection from ('127.0.0.1', 54304)
default SAC action


In [12]:
algo.config = new_config

In [13]:
episode_reward = 0
terminated = truncated = False
obs, info = env.reset()

while not terminated and not truncated:
    action = algo.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward

reset triggered
reload save


AttributeError: 'dict' object has no attribute 'normalize_actions'

c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 56.27680129999999
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


Try using savepoints

In [22]:
from ray.rllib.algorithms.algorithm import Algorithm
agent = Algorithm.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-13_00-31-13_kmaa6ii/checkpoint_000161")

2023-05-13 23:15:03,727	INFO algorithm.py:506 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2023-05-13 23:15:06,293	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8266 
2023-05-13 23:15:13,439	ERROR actor_manager.py:496 -- Ray error, taking actor 1 out of service. The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=21412, ip=127.0.0.1, repr=<ray.rllib.evaluation.rollout_worker.RolloutWorker object at 0x00000254018892E0>)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\gymnasium\envs\registration.py", line 569, in make
    _check_version_exists(ns, name, version)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\gymnasium\envs\registration.py", line 219, in _check_version_exists
    _check_name_exists(ns, name)
  File "c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\gymnasium\envs\registration.py", line 

EnvError: The env string you provided ('gt-rtgym-env-v1') is:
a) Not a supported/installed environment.
b) Not a tune-registered environment creator.
c) Not a valid env class string.

Try one of the following:
a) For Atari support: `pip install gym[atari] autorom[accept-rom-license]`.
   For VizDoom support: Install VizDoom
   (https://github.com/mwydmuch/ViZDoom/blob/master/doc/Building.md) and
   `pip install vizdoomgym`.
   For PyBullet support: `pip install pybullet`.
b) To register your custom env, do `from ray import tune;
   tune.register('[name]', lambda cfg: [return env obj from here using cfg])`.
   Then in your config, do `config['env'] = [name]`.
c) Make sure you provide a fully qualified classpath, e.g.:
   `ray.rllib.examples.env.repeat_after_me_env.RepeatAfterMeEnv`


In [19]:
from ray.rllib.policy.policy import Policy

# env = gymnasium.make("real-time-gym-v1")
my_restored_policy = Policy.from_checkpoint("C:/Users/nadir/ray_results/PPO_gt-rtgym-env-v1_2023-05-13_00-31-13_kmaa6ii/checkpoint_000161")

print("next")



2023-05-13 23:10:41,887	INFO policy.py:1214 -- Policy (worker=local) running on CPU.
2023-05-13 23:10:41,887	INFO torch_policy_v2.py:110 -- Found 1 visible cuda devices.


next


In [3]:
ray.init()

2023-05-13 22:59:01,352	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.9
Ray version:,2.3.0
Dashboard:,http://127.0.0.1:8265


In [4]:
env = gymnasium.make("real-time-gym-v1", config=my_config)

GT Real Time instantiated
GT AI Server instantiated for rtgym
starting up on localhost port 9999
Waiting for a connection
Connection from ('127.0.0.1', 51390)
default SAC action


In [20]:
episode_reward = 0
terminated = truncated = False
obs, info = env.reset()

reset triggered
reload save


c:\Users\nadir\anaconda3\envs\GTAI2\lib\site-packages\rtgym\envs\real_time_env.py:376: UserWarning: Time-step timed out. Elapsed since last time-step: 354.792402
  warnings.warn(f"Time-step timed out. Elapsed since last time-step: {now - self.__t_end}")


In [21]:
while not terminated and not truncated:
    action = my_restored_policy.compute_single_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    episode_reward += reward

AttributeError: 'dict' object has no attribute 'compute_single_action'